# Now we are going to use LangChain with our Model

LangChain integrates with SageMaker. Documentation for LangChain cam be found at https://python.langchain.com/en/latest/getting_started/getting_started.html

Please select the Data Science 3.0 Kernel to run this Notebook

In [ ]:
%%capture
!pip install langchain

In [ ]:
import boto3
import json
from typing import Dict
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.docstore.document import Document

region = boto3.session.Session().region_name

sm_endpoint_name = "TurkuNLP-gpt3-finnish-3B"

In [ ]:
example_doc_1 = """
HUUHKAJAT otti perjantaina loppuunmyydyllä Olympiastadionilla yli 32 000 katsojan edessä karsintojen ensimmäisen kotivoiton. Se tuli vahvalla esityksellä, jossa avaintekijöitä olivat vastustajan vahvuuksien eliminointi ja Suomen hyökkäyspelin vaarallisuus.

Suomen kaikkien aikojen maalintekijästä Teemu Pukista on kehkeytynyt syöttökone näissä karsinnoissa. Hän syötti Joel Pohjanpalon maalin avausjaksolla ja Oliver Antmanin 2–0-johtomaalin toisella jaksolla.

ENSIMMÄINEN minuutti näytti jo hyvin, mistä pelissä on kyse. Slovenia pyrki pelaamaan palloa ilmassa nopeasti hyökkääjilleen, ja Suomi pyrki hyökätessään hajottamaan vieraiden puolustusmuotoa ja pelaamaan palloa lyhyillä syötöillä eteenpäin.

Suomi näytti heti siltä, että se pystyisi uhkaamaan Slovenian puolustusta. Ongelmana vain olivat alun syöttöharhat, kun Huuhkajat menetti pallon useampaan otteeseen alussa heikkojen syöttöjen takia.
"""

docs = [
    Document(
        page_content=example_doc_1,
    )
]

In [ ]:
query = """Kuka voitti pelin?"""

prompt_template = """Käytä seuraavia kontekstin osia vastataksesi lopussa olevaan kysymykseen.

{context}

Kysymys: {question}
Vastaus: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

content_handler = ContentHandler()

In [ ]:
chain = load_qa_chain(
    llm=SagemakerEndpoint(
        endpoint_name=sm_endpoint_name, 
        region_name=region,
        model_kwargs={"temperature":0.1, "max_new_tokens": 10},
        content_handler=content_handler
    ),
    prompt=PROMPT
)

In [ ]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)



## HTTP request chain

In [ ]:
template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [ ]:
chain = LLMRequestsChain(
    llm_chain=LLMChain(
        llm=SagemakerEndpoint(
            endpoint_name=sm_endpoint_name, 
            region_name=region,
            model_kwargs={"temperature":0.1, "max_new_tokens": 10},
            content_handler=content_handler),
        prompt=PROMPT
    )
)

In [ ]:
question = "What are the Three (3) biggest countries, and their respective sizes?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}

In [ ]:
chain(inputs)